In [1]:
import json
from collections import Counter
import bz2
import regex
from tqdm import tqdm
from scipy import sparse
import pickle

In [2]:
import pandas as pd
import numpy as np
import nltk

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
### загрузим лемматизированный текст с первого ДЗ

In [3]:
df_responses = pd.read_json('df_responses.json')

In [4]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
#№ nltk.download('punkt')

In [14]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x1f00f005cd0>)

In [5]:
# преобразуем текст в список слов
%%time
df_responses['text2'] = df_responses.text.str.split()

Wall time: 4.92 s


In [6]:
# преобразуем text2 из датафрейма в список
sentences = df_responses['text2'].to_list()

### Обучим по коллекции предложений word2vec

In [8]:
print("Training model...")

%time model_ru = word2vec.Word2Vec(sentences, workers=4, min_count=10, \
                                       window=10, sample=1e-3)

Training model...
Wall time: 2min 12s


In [9]:
model_ru.save('model_ru.model')

### 1. Приведем несколько удачных и неудачных примеров решения стандартных текстов для word2vec:

#### тест на определение ближайших слов

In [84]:
words = poz_l
for word in words:

    word = word + '_S'
    if word in model_ru.wv:
        print(word)
        for word, sim in model_ru.wv.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')

In [88]:
print(model_ru.wv.most_similar("банк", topn=3))
print(model_ru.wv.most_similar("кредит", topn=3))
print(model_ru.wv.most_similar("депозит", topn=3))
print(model_ru.wv.most_similar("карта", topn=3))

[('банка', 0.8122317790985107), ('мкб', 0.4765799641609192), ('банкинг', 0.4307345151901245)]
[('автокредит', 0.7700268030166626), ('потребкредит', 0.7505057454109192), ('ссуда', 0.7018482685089111)]
[('вклад', 0.9231784343719482), ('доходный', 0.6512096524238586), ('депозитный', 0.6486349105834961)]
[('карточка', 0.8131100535392761), ('неименной', 0.6556129455566406), ('именной', 0.634346604347229)]


#### тест на аналогии (мужчина – король : женщина – королева)

In [93]:
print('банк', 'отделение', ':', model_ru.wv.similarity('банк', 'отделение'))
print('кредит', 'депозит', ':', model_ru.wv.similarity('кредит', 'депозит'))
print('кредит', 'мошенничество', ':', model_ru.wv.similarity('кредит', 'мошенничество'))
print('депозит', 'мошенничество', ':', model_ru.wv.similarity('депозит', 'мошенничество'))
print('карта', 'мошенничество', ':', model_ru.wv.similarity('карта', 'мошенничество'))
print('карта', 'мошенничество', ':', model_ru.wv.similarity('карта', 'мошенничество'))
print('сбербанк', 'банк', ':', model_ru.wv.similarity('сбербанк', 'банк'))
print('втб', 'банк', ':', model_ru.wv.similarity('втб', 'банк'))
print('сбербанк', 'втб', ':', model_ru.wv.similarity('сбербанк', 'втб'))

банк отделение : 0.13544963
кредит депозит : 0.084522225
кредит мошенничество : -0.062804975
депозит мошенничество : -0.20968243
карта мошенничество : 0.05863786
карта мошенничество : 0.05863786
сбербанк банк : 0.3259032
втб банк : 0.31949484
сбербанк втб : 0.33785897


#### тест на определение лишнего слова.

In [97]:
print(model_ru.wv.doesnt_match(['банк', 'кредит', 'депозит', 'дверь']))

дверь


In [98]:
print(model_ru.wv.doesnt_match(['карта', 'кредит', 'депозит', 'телефон']))

телефон


In [99]:
print(model_ru.wv.doesnt_match(['кредит', 'доверие', 'мошенничество']))

кредит


### 2. Постройте несколько визуализаций:

In [22]:
from nltk import FreqDist
from tqdm import tqdm
from sklearn.manifold import TSNE

top_words = []


fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(10000):
    top_words.append(w[0])
    
print(top_words[:50:])
top_words_vec = model_ru.wv[top_words]

100%|████████████████████████████████████████████████████████████████████████| 153499/153499 [00:20<00:00, 7419.82it/s]


['банк', 'карта', 'деньги', 'день', 'кредит', 'отделение', 'весь', 'клиент', 'сотрудник', 'счет', 'сказать', 'сумма', 'получать', 'заявление', 'год', 'вопрос', 'время', 'кредитный', 'приходить', 'ответ', 'вклад', 'офис', 'телефон', 'данный', 'платеж', 'договор', 'ваш', 'номер', 'звонить', 'банкомат', 'сбербанк', 'очень', 'позвонить', 'г', 'месяц', 'написать', 'проблема', 'обращаться', 'решать', 'работать', 'средство', 'рубль', 'работа', 'документ', 'ситуация', 'звонок', 'отвечать', 'говорить', 'человек', 'хотеть']


In [30]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne_100 = tsne.fit_transform(top_words_vec[:100])
top_words_tsne_500 = tsne.fit_transform(top_words_vec[:500])

Wall time: 5.26 s


#### TSNE для топ-100 

In [116]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (ru model, top100 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne_100[:,0],
                                    x2=top_words_tsne_100[:,1],
                                    names=top_words[:100]))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

Осмысленные кластеры слов: документ-договор, банкомат-снимать, сотрудник-оператор-девушка, карта-счет, год-месяц

#### TSNE для топ-500 

In [117]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (ru model, top500 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne_500[:,0],
                                    x2=top_words_tsne_500[:,1],
                                    names=top_words[:500]))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [ ]:
Осмысленные кластеры слов: названия месяцев, ежемесячный-график, обслуживание-сервис-услуга, 
    представитель-работник-сотрудник, очередь-сотрудник

#### зададим координаты для нового пространства следующим образом: одна ось описывает отношение "плохо – хорошо", вторая – "медленно – быстро" и найдем координаты названий банков в этих координатах. 

In [45]:
# получим индексы банков
banks = ['сбербанк', 'втб', 'росбанк', 'мкб', 'авангард', 'отп']
for bank in banks:
    try:
        print(bank, top_words.index(bank))
    except:
        print(f'{bank} is not in list')

сбербанк 30
втб 238
росбанк 964
мкб 814
авангард 772
отп 913


In [18]:
# получим индексы значений
props = ['хороший', 'плохо', 'медленно','быстро']
for prop in props:
    print(prop, top_words.index(prop))

хороший 192
плохо 802
медленно 2057
быстро 199


In [48]:
# хорошо - плохо
x = top_words_vec[192]-top_words_vec[802]

# медленно  - быстро
y = top_words_vec[2057]-top_words_vec[199]

In [49]:
# сбербанк
sber_x = top_words_vec[30] * x
sber_y = top_words_vec[30] * y

# vtb
vtb_x = top_words_vec[238] * x
vtb_y = top_words_vec[238] * y

# росбанк
ros_x = top_words_vec[964] * x
ros_y = top_words_vec[964] * y

# мкб
mkb_x = top_words_vec[814] * x
mkb_y = top_words_vec[814] * y

# авангард
avg_x = top_words_vec[772] * x
avg_y = top_words_vec[772] * y

# отп
otp_x = top_words_vec[913] * x
otp_y = top_words_vec[913] * y


In [50]:
banks_x = [sber_x, vtb_x, ros_x, mkb_x, avg_x, otp_x]
banks_y = [sber_y, vtb_y, ros_y, mkb_y, avg_y, otp_y]

In [51]:
tsne = TSNE(n_components=1, random_state=0)
top_words_tsne_banks_x = tsne.fit_transform(banks_x)
top_words_tsne_banks_y = tsne.fit_transform(banks_y)

In [52]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (ru model, banks)")

source = ColumnDataSource(data=dict(x1=top_words_tsne_banks_x,
                                    x2=top_words_tsne_banks_y,
                                    names=banks))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

In [ ]:
МКБ обслуживает медленно и плохо
ОТП - быстро и хорошо
СБЕР, ВТБ - средне